# Prepare Chicago Crime Data for a GitHub Repository

- Original Notebook Source: https://github.com/coding-dojo-data-science/preparing-chicago-crime-data
- Updated 11/17/22

>- This notebook will process a "Crimes - 2001 to Preset.csv" crime file in your Downloads folder and save it as smaller .csv's in a new "Data/Chicago/" folder inside this notebook's folder/repo.

# INSTRUCTIONS

- 1) Go to the Chicago Data Portal's page for ["Crimes - 2001 to Preset"](https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-Present/ijzp-q8t2).

- 2) Click on the Export button on the top right and select CSV. 
    - Save the file to your Downloads folder instead of your repository. **The file is too big for a repository.**
    
    
    
- 3) Wait for the full file to download. 
    - It is very large (over >1.7GB and may take several minutes to fully download.)
    
    
- 4) Once the download is complete, change `RAW_FILE` variable below to match the filepath to the downloaded file.

## 🚨 Set the correct `RAW_FILE` path

- The cell below will attempt to check your Downloads folder for any file with a name that contains "Crimes_-_2001_to_Present".
    - If you know the file path already, you can skip the next cell and just manually set the RAW_FILE variable in the following code cell.

In [1]:
## Run the cell below to attempt to programmatically find your crime file
import os,glob

## Getting the home folder from environment variables
home_folder = os.environ['HOME']
# print("- Your Home Folder is: " + home_folder)

## Check for downloads folder
if 'Downloads' in os.listdir(home_folder):
    
    
    # Print the Downloads folder path
    dl_folder = os.path.abspath(os.path.join(home_folder,'Downloads'))
    print(f"- Your Downloads folder is '{dl_folder}/'\n")
    
    ## checking for crime files using glob
    crime_files = sorted(glob.glob(dl_folder+'/**/Crimes_-_2001_to_Present*',recursive=True))
    
    # If more than 
    if len(crime_files)==1:
        RAW_FILE = crime_files[0]
        
    elif len(crime_files)>1:
        print('[i] The following files were found:')
        
        for i, fname in enumerate(crime_files):
            print(f"\tcrime_files[{i}] = '{fname}'")
        print(f'\n- Please fill in the RAW_FILE variable in the code cell below with the correct filepath.')

else:
    print(f'[!] Could not programmatically find your downloads folder.')
    print('- Try using Finder (on Mac) or File Explorer (Windows) to navigate to your Downloads folder.')


- Your Downloads folder is '/Users/codingdojo/Downloads/'

[i] The following files were found:
	crime_files[0] = '/Users/codingdojo/Downloads/Crimes_-_2001_to_Present (1).csv'
	crime_files[1] = '/Users/codingdojo/Downloads/Crimes_-_2001_to_Present (2).csv'
	crime_files[2] = '/Users/codingdojo/Downloads/Crimes_-_2001_to_Present (3).csv'
	crime_files[3] = '/Users/codingdojo/Downloads/Crimes_-_2001_to_Present.csv'

- Please fill in the RAW_FILE variable in the code cell below with the correct filepath.


In [2]:
## (Required) MAKE SURE TO CHANGE THIS VARIABLE TO MATCH YOUR LOCAL FILE NAME
RAW_FILE = crime_files[2] #(or slice correct index from the crime_files list)
RAW_FILE

'/Users/codingdojo/Downloads/Crimes_-_2001_to_Present (3).csv'

In [3]:
## (Optional) SET THE FOLDER FOR FINAL FILES
OUTPUT_FOLDER = 'Data/Chicago/'
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# 🔄 Full Workflow

- Now that your RAW_FILE variable is set either:
    - On the toolbar, click on the Kernel menu > "Restart and Run All".
    - OR click on this cell first, then on the toolbar click on the "Cell" menu > "Run All Below"

In [4]:
import pandas as pd
pd.set_option('display.max_columns', 100)
pd.set_option('display.float_format',lambda x: f"{x:,.2f}")

In [5]:
chicago_full = pd.read_csv(RAW_FILE)
chicago_full

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,10224738,HY411648,09/05/2015 01:30:00 PM,043XX S WOOD ST,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,924,9.00,12.00,61.00,08B,"1,165,074.00","1,875,917.00",2015,02/10/2018 03:50:01 PM,41.82,-87.67,"(41.815117282, -87.669999562)"
1,10224739,HY411615,09/04/2015 11:30:00 AM,008XX N CENTRAL AVE,0870,THEFT,POCKET-PICKING,CTA BUS,False,False,1511,15.00,29.00,25.00,06,"1,138,875.00","1,904,869.00",2015,02/10/2018 03:50:01 PM,41.90,-87.77,"(41.895080471, -87.765400451)"
2,11646166,JC213529,09/01/2018 12:01:00 AM,082XX S INGLESIDE AVE,0810,THEFT,OVER $500,RESIDENCE,False,True,631,6.00,8.00,44.00,06,NaN,NaN,2018,04/06/2019 04:04:43 PM,NaN,NaN,NaN
3,10224740,HY411595,09/05/2015 12:45:00 PM,035XX W BARRY AVE,2023,NARCOTICS,POSS: HEROIN(BRN/TAN),SIDEWALK,True,False,1412,14.00,35.00,21.00,18,"1,152,037.00","1,920,384.00",2015,02/10/2018 03:50:01 PM,41.94,-87.72,"(41.937405765, -87.716649687)"
4,10224741,HY411610,09/05/2015 01:00:00 PM,0000X N LARAMIE AVE,0560,ASSAULT,SIMPLE,APARTMENT,False,True,1522,15.00,28.00,25.00,08A,"1,141,706.00","1,900,086.00",2015,02/10/2018 03:50:01 PM,41.88,-87.76,"(41.881903443, -87.755121152)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7672793,12847575,JF420478,09/01/2022 05:00:00 AM,005XX W SURF ST,2825,OTHER OFFENSE,HARASSMENT BY TELEPHONE,RESIDENCE,False,True,1934,19.00,44.00,6.00,26,"1,172,497.00","1,919,410.00",2022,11/12/2022 03:46:21 PM,41.93,-87.64,"(41.934304581, -87.641484982)"
7672794,12847801,JF420319,07/08/2022 12:00:00 AM,114XX S PRAIRIE AVE,1130,DECEPTIVE PRACTICE,FRAUD OR CONFIDENCE GAME,STREET,False,False,531,5.00,9.00,49.00,11,"1,179,966.00","1,828,818.00",2022,11/12/2022 03:46:21 PM,41.69,-87.62,"(41.685543881, -87.616812541)"
7672795,12847324,JF420102,09/27/2022 11:00:00 AM,023XX E 70TH ST,0810,THEFT,OVER $500,RESIDENCE,False,False,331,3.00,5.00,43.00,06,"1,193,181.00","1,859,005.00",2022,11/12/2022 03:46:21 PM,41.77,-87.57,"(41.768068052, -87.567452932)"
7672796,12847570,JF420427,09/03/2022 10:25:00 AM,052XX W CARMEN AVE,2021,NARCOTICS,POSSESS - BARBITURATES,RESIDENCE - YARD (FRONT / BACK),True,False,1623,16.00,45.00,11.00,18,"1,140,553.00","1,933,418.00",2022,11/12/2022 03:46:21 PM,41.97,-87.76,"(41.973391184, -87.758534512)"


In [6]:
# this cell can take a minute + to run
chicago_full['Datetime'] = pd.to_datetime(chicago_full['Date'])
chicago_full = chicago_full.sort_values('Datetime')
chicago_full = chicago_full.set_index('Datetime')
chicago_full

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
Datetime,,,,,,,,,,,,,,,,,,,,,,
2001-01-01 00:00:00,2717983,HJ347945,01/01/2001 12:00:00 AM,058XX W DIVISION ST,1751,OFFENSE INVOLVING CHILDREN,CRIM SEX ABUSE BY FAM MEMBER,RESIDENCE,False,False,1511,15.00,29.00,25.00,20,"1,137,142.00","1,907,358.00",2001,08/17/2015 03:03:40 PM,41.90,-87.77,"(41.9019419, -87.771705566)"
2001-01-01 00:00:00,6040504,HP105033,01/01/2001 12:00:00 AM,018XX S ST LOUIS AVE,1751,OFFENSE INVOLVING CHILDREN,CRIM SEX ABUSE BY FAM MEMBER,RESIDENCE,False,False,1021,10.00,24.00,29.00,20,"1,153,282.00","1,890,985.00",2001,02/01/2008 01:05:02 AM,41.86,-87.71,"(41.856707412, -87.712855481)"
2001-01-01 00:00:00,9652075,HX302422,01/01/2001 12:00:00 AM,033XX W WALNUT ST,0266,CRIM SEXUAL ASSAULT,PREDATORY,RESIDENCE,False,False,1123,11.00,28.00,27.00,02,NaN,NaN,2001,08/17/2015 03:03:40 PM,NaN,NaN,NaN
2001-01-01 00:00:00,9754894,HX403520,01/01/2001 12:00:00 AM,031XX W DOUGLAS BLVD,1562,SEX OFFENSE,AGG CRIMINAL SEXUAL ABUSE,CHURCH/SYNAGOGUE/PLACE OF WORSHIP,False,False,1022,10.00,24.00,29.00,17,NaN,NaN,2001,08/17/2015 03:03:40 PM,NaN,NaN,NaN
2001-01-01 00:00:00,1692462,G485012,01/01/2001 12:00:00 AM,063XX S CENTRAL AV,2820,OTHER OFFENSE,TELEPHONE THREAT,OTHER,False,False,813,8.00,NaN,NaN,26,"1,140,208.00","1,862,044.00",2001,08/17/2015 03:03:40 PM,41.78,-87.76,"(41.777537944, -87.761551613)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-08 23:45:00,12887106,JF467817,11/08/2022 11:45:00 PM,019XX E 73RD PL,0910,MOTOR VEHICLE THEFT,AUTOMOBILE,STREET,False,False,333,3.00,7.00,43.00,07,"1,190,612.00","1,856,624.00",2022,11/15/2022 03:50:01 PM,41.76,-87.58,"(41.761596782, -87.57694609)"
2022-11-08 23:45:00,12891028,JF471668,11/08/2022 11:45:00 PM,004XX N WOLCOTT AVE,0810,THEFT,OVER $500,STREET,False,False,1221,12.00,27.00,24.00,06,"1,163,653.00","1,903,249.00",2022,11/15/2022 03:50:01 PM,41.89,-87.67,"(41.890148934, -87.674442715)"
2022-11-08 23:48:00,12887011,JF467715,11/08/2022 11:48:00 PM,014XX N LARAMIE AVE,051A,ASSAULT,AGGRAVATED - HANDGUN,STREET,False,False,2533,25.00,37.00,25.00,04A,"1,141,438.00","1,909,167.00",2022,11/15/2022 03:50:01 PM,41.91,-87.76,"(41.906827741, -87.755880834)"


In [7]:
(chicago_full.isna().sum()/len(chicago_full)).round(2)

ID                     0.00
Case Number            0.00
Date                   0.00
Block                  0.00
IUCR                   0.00
Primary Type           0.00
Description            0.00
Location Description   0.00
Arrest                 0.00
Domestic               0.00
Beat                   0.00
District               0.00
Ward                   0.08
Community Area         0.08
FBI Code               0.00
X Coordinate           0.01
Y Coordinate           0.01
Year                   0.00
Updated On             0.00
Latitude               0.01
Longitude              0.01
Location               0.01
dtype: float64

## Separate the Full Dataset by Years

### Creating Bins for Files

In [8]:
# save the years for every crime
years = chicago_full.index.year
years

Int64Index([2001, 2001, 2001, 2001, 2001, 2001, 2001, 2001, 2001, 2001,
            ...
            2022, 2022, 2022, 2022, 2022, 2022, 2022, 2022, 2022, 2022],
           dtype='int64', name='Datetime', length=7672798)

In [9]:
# calculate 5-year bins
year_bins = (chicago_full.index.year-2000)//5
year_bins

Int64Index([0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
            ...
            4, 4, 4, 4, 4, 4, 4, 4, 4, 4],
           dtype='int64', name='Datetime', length=7672798)

In [10]:
# # Fill in a new file_bin column
# chicago_full['file_bin'] = year_bins
# chicago_full['file_bin'].value_counts(dropna=False)

In [11]:
# Fill in a new file_bin column
chicago_full['file_bin'] = years#_bins
chicago_full['file_bin'].value_counts(dropna=False)

2002    486794
2001    485854
2003    475977
2004    469417
2005    453762
2006    448169
2007    437077
2008    427157
2009    392817
2010    370483
2011    351948
2012    336246
2013    307444
2014    275712
2016    269758
2017    269003
2018    268687
2015    264708
2019    261146
2020    211936
2021    207946
2022    200757
Name: file_bin, dtype: int64

In [12]:
## Dropping unneeded columns to reduce file size
drop_cols = ["X Coordinate","Y Coordinate", "Community Area","FBI Code",
             "Case Number","Updated On",'Block','Location','IUCR']

In [13]:
# ## columns to keep
# keep_cols = list(chicago_full.drop(columns=['file_bin',*drop_cols]).columns)
# keep_cols

In [14]:
chicago_final = chicago_full.drop(columns=drop_cols).sort_index().reset_index()
chicago_final

,Datetime,ID,Date,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Year,Latitude,Longitude,file_bin
0,2001-01-01 00:00:00,2717983,01/01/2001 12:00:00 AM,OFFENSE INVOLVING CHILDREN,CRIM SEX ABUSE BY FAM MEMBER,RESIDENCE,False,False,1511,15.00,29.00,2001,41.90,-87.77,2001
1,2001-01-01 00:00:00,6040504,01/01/2001 12:00:00 AM,OFFENSE INVOLVING CHILDREN,CRIM SEX ABUSE BY FAM MEMBER,RESIDENCE,False,False,1021,10.00,24.00,2001,41.86,-87.71,2001
2,2001-01-01 00:00:00,9652075,01/01/2001 12:00:00 AM,CRIM SEXUAL ASSAULT,PREDATORY,RESIDENCE,False,False,1123,11.00,28.00,2001,NaN,NaN,2001
3,2001-01-01 00:00:00,9754894,01/01/2001 12:00:00 AM,SEX OFFENSE,AGG CRIMINAL SEXUAL ABUSE,CHURCH/SYNAGOGUE/PLACE OF WORSHIP,False,False,1022,10.00,24.00,2001,NaN,NaN,2001
4,2001-01-01 00:00:00,1692462,01/01/2001 12:00:00 AM,OTHER OFFENSE,TELEPHONE THREAT,OTHER,False,False,813,8.00,NaN,2001,41.78,-87.76,2001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7672793,2022-11-08 23:45:00,12887106,11/08/2022 11:45:00 PM,MOTOR VEHICLE THEFT,AUTOMOBILE,STREET,False,False,333,3.00,7.00,2022,41.76,-87.58,2022
7672794,2022-11-08 23:45:00,12891028,11/08/2022 11:45:00 PM,THEFT,OVER $500,STREET,False,False,1221,12.00,27.00,2022,41.89,-87.67,2022
7672795,2022-11-08 23:48:00,12887011,11/08/2022 11:48:00 PM,ASSAULT,AGGRAVATED - HANDGUN,STREET,False,False,2533,25.00,37.00,2022,41.91,-87.76,2022
7672796,2022-11-08 23:52:00,12887763,11/08/2022 11:52:00 PM,OTHER OFFENSE,VIOLATE ORDER OF PROTECTION,APARTMENT,False,True,212,2.00,4.00,2022,41.83,-87.61,2022


In [15]:
chicago_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7672798 entries, 0 to 7672797
Data columns (total 15 columns):
 #   Column                Dtype         
---  ------                -----         
 0   Datetime              datetime64[ns]
 1   ID                    int64         
 2   Date                  object        
 3   Primary Type          object        
 4   Description           object        
 5   Location Description  object        
 6   Arrest                bool          
 7   Domestic              bool          
 8   Beat                  int64         
 9   District              float64       
 10  Ward                  float64       
 11  Year                  int64         
 12  Latitude              float64       
 13  Longitude             float64       
 14  file_bin              int64         
dtypes: bool(2), datetime64[ns](1), float64(4), int64(4), object(4)
memory usage: 775.6+ MB


In [16]:
# chicago_final = chicago_full[keep_cols].sort_index().reset_index()
# chicago_final

In [17]:
chicago_final.memory_usage(deep=True).astype(float)

Index                          128.00
Datetime                61,382,384.00
ID                      61,382,384.00
Date                   606,151,042.00
Primary Type           514,228,462.00
Description            561,502,312.00
Location Description   522,901,704.00
Arrest                   7,672,798.00
Domestic                 7,672,798.00
Beat                    61,382,384.00
District                61,382,384.00
Ward                    61,382,384.00
Year                    61,382,384.00
Latitude                61,382,384.00
Longitude               61,382,384.00
file_bin                61,382,384.00
dtype: float64

In [18]:
# chicago_final.isna().sum()

In [19]:
from tqdm.notebook import tqdm
import os

In [20]:
# unique # of year bins
file_bins = chicago_final['file_bin'].unique()
file_bins

array([2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011,
       2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022])

In [21]:
FINAL_DROP = ['Datetime','file_bin']#,'Location Description']

In [22]:
## set save location 
os.makedirs(OUTPUT_FOLDER, exist_ok=True)


for curr_bin in tqdm(file_bins):
    ## save temp slices of dfs to save.
    temp_df = chicago_final.loc[ chicago_final['file_bin']==curr_bin]
    temp_df = temp_df.drop(columns=FINAL_DROP)

    ## get years for filename
#     start = temp_df.index.year.min()
#     start = temp_df['Datetime'].dt.year.min()
# #     end = temp_df.index.year.max()
#     end = temp_df['Datetime'].dt.year.max()
    
#     fname_temp = f"{OUTPUT_FOLDER}Chicago-Crime_{start}-{end}.csv"#.gz
    fname_temp = f"{OUTPUT_FOLDER}Chicago-Crime_{curr_bin}.csv"#.gz
    temp_df.to_csv(fname_temp,index=False) #compression='gzip'

    print(fname_temp)

  0%|          | 0/22 [00:00<?, ?it/s]

Data/Chicago/Chicago-Crime_2001.csv
Data/Chicago/Chicago-Crime_2002.csv
Data/Chicago/Chicago-Crime_2003.csv
Data/Chicago/Chicago-Crime_2004.csv
Data/Chicago/Chicago-Crime_2005.csv
Data/Chicago/Chicago-Crime_2006.csv
Data/Chicago/Chicago-Crime_2007.csv
Data/Chicago/Chicago-Crime_2008.csv
Data/Chicago/Chicago-Crime_2009.csv
Data/Chicago/Chicago-Crime_2010.csv
Data/Chicago/Chicago-Crime_2011.csv
Data/Chicago/Chicago-Crime_2012.csv
Data/Chicago/Chicago-Crime_2013.csv
Data/Chicago/Chicago-Crime_2014.csv
Data/Chicago/Chicago-Crime_2015.csv
Data/Chicago/Chicago-Crime_2016.csv
Data/Chicago/Chicago-Crime_2017.csv
Data/Chicago/Chicago-Crime_2018.csv
Data/Chicago/Chicago-Crime_2019.csv
Data/Chicago/Chicago-Crime_2020.csv
Data/Chicago/Chicago-Crime_2021.csv
Data/Chicago/Chicago-Crime_2022.csv


In [23]:
import glob
saved_files = sorted(glob.glob(OUTPUT_FOLDER+'*.*csv'))
saved_files

['Data/Chicago/Chicago-Crime_2001.csv',
 'Data/Chicago/Chicago-Crime_2002.csv',
 'Data/Chicago/Chicago-Crime_2003.csv',
 'Data/Chicago/Chicago-Crime_2004.csv',
 'Data/Chicago/Chicago-Crime_2005.csv',
 'Data/Chicago/Chicago-Crime_2006.csv',
 'Data/Chicago/Chicago-Crime_2007.csv',
 'Data/Chicago/Chicago-Crime_2008.csv',
 'Data/Chicago/Chicago-Crime_2009.csv',
 'Data/Chicago/Chicago-Crime_2010.csv',
 'Data/Chicago/Chicago-Crime_2011.csv',
 'Data/Chicago/Chicago-Crime_2012.csv',
 'Data/Chicago/Chicago-Crime_2013.csv',
 'Data/Chicago/Chicago-Crime_2014.csv',
 'Data/Chicago/Chicago-Crime_2015.csv',
 'Data/Chicago/Chicago-Crime_2016.csv',
 'Data/Chicago/Chicago-Crime_2017.csv',
 'Data/Chicago/Chicago-Crime_2018.csv',
 'Data/Chicago/Chicago-Crime_2019.csv',
 'Data/Chicago/Chicago-Crime_2020.csv',
 'Data/Chicago/Chicago-Crime_2021.csv',
 'Data/Chicago/Chicago-Crime_2022.csv']

In [24]:
## create a README.txt for the zip files
readme = """Source URL: 
- https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-Present/ijzp-q8t2
- Filtered for years 2000-Present.

Downloaded 07/18/2022
- Files are split into 1 year per file.

EXAMPLE USAGE:
>> import glob
>> import pandas as pd
>> folder = "Data/Chicago/"
>> crime_files = sorted(glob.glob(folder+"*.csv"))
>> df = pd.concat([pd.read_csv(f) for f in crime_files])
"""
print(readme)


with open(f"{OUTPUT_FOLDER}README.txt",'w') as f:
    f.write(readme)

Source URL: 
- https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-Present/ijzp-q8t2
- Filtered for years 2000-Present.

Downloaded 07/18/2022
- Files are split into 1 year per file.

EXAMPLE USAGE:
>> import glob
>> import pandas as pd
>> folder = "Data/Chicago/"
>> crime_files = sorted(glob.glob(folder+"*.csv"))
>> df = pd.concat([pd.read_csv(f) for f in crime_files])



## Confirmation

- Follow the example usage above to test if your files were created successfully.

In [25]:
import glob
import pandas as pd
crime_files = sorted(glob.glob(OUTPUT_FOLDER+"*.csv"))
df = pd.concat([pd.read_csv(f) for f in crime_files])
df

,ID,Date,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Year,Latitude,Longitude
0,2717983,01/01/2001 12:00:00 AM,OFFENSE INVOLVING CHILDREN,CRIM SEX ABUSE BY FAM MEMBER,RESIDENCE,False,False,1511,15.00,29.00,2001,41.90,-87.77
1,6040504,01/01/2001 12:00:00 AM,OFFENSE INVOLVING CHILDREN,CRIM SEX ABUSE BY FAM MEMBER,RESIDENCE,False,False,1021,10.00,24.00,2001,41.86,-87.71
2,9652075,01/01/2001 12:00:00 AM,CRIM SEXUAL ASSAULT,PREDATORY,RESIDENCE,False,False,1123,11.00,28.00,2001,NaN,NaN
3,9754894,01/01/2001 12:00:00 AM,SEX OFFENSE,AGG CRIMINAL SEXUAL ABUSE,CHURCH/SYNAGOGUE/PLACE OF WORSHIP,False,False,1022,10.00,24.00,2001,NaN,NaN
4,1692462,01/01/2001 12:00:00 AM,OTHER OFFENSE,TELEPHONE THREAT,OTHER,False,False,813,8.00,NaN,2001,41.78,-87.76
...,...,...,...,...,...,...,...,...,...,...,...,...,...
200752,12887106,11/08/2022 11:45:00 PM,MOTOR VEHICLE THEFT,AUTOMOBILE,STREET,False,False,333,3.00,7.00,2022,41.76,-87.58
200753,12891028,11/08/2022 11:45:00 PM,THEFT,OVER $500,STREET,False,False,1221,12.00,27.00,2022,41.89,-87.67
200754,12887011,11/08/2022 11:48:00 PM,ASSAULT,AGGRAVATED - HANDGUN,STREET,False,False,2533,25.00,37.00,2022,41.91,-87.76
200755,12887763,11/08/2022 11:52:00 PM,OTHER OFFENSE,VIOLATE ORDER OF PROTECTION,APARTMENT,False,True,212,2.00,4.00,2022,41.83,-87.61
